In [9]:
library(tidyverse)
library(ggpubr)
library(GGally)
library(yaml)
library(patchwork)
#library(Seurat)
#library(ggvenn)

#read config
config <- read_yaml(file = "../CHARM//config.yaml")

#Common
Raw <- read_table2("../stat/raw.fq.stat",col_names=F) %>% arrange(X1) %>% rowwise() %>% mutate(X2 = X2/4*300/1000000000,X1 = strsplit(X1,split = "/")[[1]][3],X1 = strsplit(X1,split = "_")[[1]][1])
DNA <- read_table2("../stat/dna.fq.stat",col_names=F) %>% arrange(X1) %>% rowwise()%>% mutate(X2 = X2/4*300/1000000000,X1 = strsplit(X1,split = "/")[[1]][3])
RNA <- read_table2("../stat/rna.fq.stat",col_names=F) %>% arrange(X1) %>% rowwise() %>% mutate(X2 = X2/4*300/1000000000,X1 = strsplit(X1,split = "/")[[1]][3])

raw_pairs <- read_table2("../stat/raw.pairs.stat",col_names=F) %>% arrange(X1) %>% rowwise() %>% mutate(X1 = strsplit(X1,split = "/")[[1]][3])
pairs_dedup <- read_table2("../stat/pairs.dedup.stat",col_names=F) %>% arrange(X1) %>% rowwise() %>% mutate(X1 = strsplit(X1,split = "/")[[1]][3])

pairs_c1 <- read_table2("../stat/pairs.c1.stat",col_names=F)%>% arrange(X1) %>%rowwise() %>% mutate(X1 = strsplit(X1,split = "/")[[1]][5]) %>% mutate(X1 = str_replace(X1,".pairs.gz",""))
pairs_c12 <- read_table2("../stat/pairs.c12.stat",col_names=F)%>% arrange(X1) %>%rowwise() %>% mutate(X1 = strsplit(X1,split = "/")[[1]][5]) %>% mutate(X1 = str_replace(X1,".pairs.gz",""))
pairs_c123 <- read_table2("../stat/pairs.c123.stat",col_names=F)%>% arrange(X1) %>%rowwise() %>% mutate(X1 = strsplit(X1,split = "/")[[1]][5]) %>% mutate(X1 = str_replace(X1,".pairs.gz",""))
inter_pairs_c123 <- read_table2(file = "../stat/inter.pairs.c123.stat",col_names = F) %>% arrange(X1) %>%rowwise() %>% mutate(X1 = strsplit(X1,split = "/")[[1]][5]) %>% mutate(X1 = str_replace(X1,".pairs.gz",""))

yperx <- read_table2("../stat/yperx.stat",col_names=F) %>% arrange(X1) %>% rowwise() %>% mutate(X1 = strsplit(X1,split = "/")[[1]][2])
stat <- Raw %>% left_join(DNA,by="X1") %>% left_join(RNA,by="X1") %>% left_join(yperx,by="X1") %>% left_join(raw_pairs,by="X1") %>% left_join(pairs_dedup,by="X1") %>% 
    left_join(pairs_c1,by="X1") %>% left_join(pairs_c12,by="X1")%>% left_join(pairs_c123,by="X1") %>% left_join(inter_pairs_c123,by="X1")
RNAres <- read_table2("../result/RNA_Res/counts.gene.total.format.tsv")
featureStat <- cbind(as.data.frame(colSums(RNAres %>% select(-gene))),colSums(RNAres %>% select(-gene) != 0)) %>% rownames_to_column("X1")
RNAresExon <- read_table2("../result/RNA_Res/counts.exon.total.format.tsv")
featureStatExon <- cbind(as.data.frame(colSums(RNAresExon %>% select(-gene))),colSums(RNAresExon %>% select(-gene) != 0))  %>% rownames_to_column("X1")

if(config$if_RNA_snp_split){
    RNAresGenome1 <- read_table2("../result/RNA_Res/counts.gene.genome1.tsv")
    RNAresGenome2 <- read_table2("../result/RNA_Res/counts.gene.genome2.tsv")
    featureStatGenome1 <- cbind(as.data.frame(colSums(RNAresGenome1 %>% select(-gene))),colSums(RNAresGenome1 %>% select(-gene) != 0)) %>% rownames_to_column("X1")
    featureStatGenome2 <- cbind(as.data.frame(colSums(RNAresGenome2 %>% select(-gene))),colSums(RNAresGenome2 %>% select(-gene) != 0)) %>% rownames_to_column("X1")
    stat <- stat %>% left_join(featureStat,by="X1") %>% left_join(featureStatExon,by="X1") %>% left_join(featureStatGenome1,by="X1")  %>% left_join(featureStatGenome2,by="X1")
    names(stat) <- c("cellname","Rawreads","DNAreads","RNAreads","yperx","raw_pairs","pairs_dedup","pairs_clean1","pairs_clean2","pairs_clean3","inter_pairs_clean3",
                     "UMIs_gene","genes_gene","UMIs_exon","genes_exon","UMIs_gene_genome1","genes_gene_genome1","UMIs_gene_genome2","genes_gene_genome2")
} else{
    stat <- stat %>% left_join(featureStat,by="X1") %>% left_join(featureStatExon,by="X1") 
    names(stat) <- c("cellname","Rawreads","DNAreads","RNAreads","yperx","raw_pairs","pairs_dedup","pairs_clean1","pairs_clean2","pairs_clean3","inter_pairs_clean3",
                     "UMIs_gene","genes_gene","UMIs_exon","genes_exon")
}


if(config$if_structure){
    rmsd <- read_table2("../stat/rmsd.info",col_names = F) %>% rowwise() %>% mutate(X1=str_split(X1,pattern = fixed("/")),cellname = X1[1], res = str_split(X1[3], fixed("."))[[1]][2],TOP3_RMSD = X5) %>% select(cellname,res,TOP3_RMSD) %>% spread(res,TOP3_RMSD) %>% arrange(cellname)
    stat <- stat %>% left_join(rmsd)
}
if (config$if_cuttag){
    reads <- tibble("cellname"=NA)
    for(i in 1:length(names(config$split))){
        temp <- read_table2(paste0("../stat/",names(config$split)[i],".read.stat"),col_names = F) %>% rowwise() %>% mutate(X1 = str_split(X1,pattern ="/")[[1]][2],X2 = X2/4*300/1000000000)
        names(temp) <- c("cellname",paste0(names(config$split)[i],"_reads"))
        reads <- reads %>% full_join(temp)
    }
    reads <- reads %>% filter(!is.na(cellname))

    frag <- tibble("cellname"=NA)
    for(i in 1:length(names(config$split))){
        temp <- read_table2(paste0("../stat/",names(config$split)[i],".frag.stat"),col_names = F) %>% rowwise() %>% mutate(X1 = str_extract(X1,pattern = "(?<=\\/)[A-Za-z0-9-]+(?=\\.)"))
        names(temp) <- c("cellname",paste0(names(config$split)[i],"_frag"))
        frag <- frag %>% full_join(temp)
    }
    frag <- frag %>% filter(!is.na(cellname))
    reads <- reads %>% left_join(frag)
    stat <- stat %>% full_join(reads)
}





── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)


── Column specification ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)


── Column specification ───────────────────

In [10]:
plot <- stat %>% mutate(RNAreadsRatio = RNAreads/(RNAreads+DNAreads),
                        pairsPerRead = raw_pairs/DNAreads/1000000000*300,
                        pairsValidRatio = pairs_clean3/raw_pairs,
                        interPairsRatio = inter_pairs_clean3/pairs_clean3) 
#plot[is.na(plot)]<-0

In [11]:
options(repr.matrix.max.cols=50, repr.matrix.max.rows=100)
plot %>% summary()

   cellname            Rawreads         DNAreads         RNAreads       
 Length:192         Min.   :0.3023   Min.   :0.2973   Min.   :0.002575  
 Class :character   1st Qu.:2.4015   1st Qu.:2.2635   1st Qu.:0.046244  
 Mode  :character   Median :4.1420   Median :3.8554   Median :0.152186  
                    Mean   :4.0641   Mean   :3.8001   Mean   :0.230230  
                    3rd Qu.:5.5827   3rd Qu.:5.1283   3rd Qu.:0.336940  
                    Max.   :8.5390   Max.   :8.1782   Max.   :1.275965  
     yperx            raw_pairs        pairs_dedup      pairs_clean1   
 Min.   :0.000965   Min.   :  65874   Min.   : 51698   Min.   : 51559  
 1st Qu.:0.005006   1st Qu.: 662960   1st Qu.:307964   1st Qu.:305150  
 Median :0.005871   Median :1322358   Median :373257   Median :368597  
 Mean   :0.007437   Mean   :1363911   Mean   :375294   Mean   :371832  
 3rd Qu.:0.006839   3rd Qu.:1912474   3rd Qu.:448309   3rd Qu.:444313  
 Max.   :0.095524   Max.   :3351396   Max.   :671798   Ma